In [ ]:
import numpy as np

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras import backend as K

from sklearn.preprocessing import StandardScaler

In [ ]:
"""Only for google colab"""
# from google.colab import drive
# drive.mount('/content/gdrive', force_remount=True)

In [ ]:
DATASET_PATH = ""
MODELS_PATH = "mlp_models/"

def load_data():
    X_train = np.load(DATASET_PATH + "X_train.npy")
    y_train = np.load(DATASET_PATH + "y_train.npy")
    X_val = np.load(DATASET_PATH + "X_val.npy")
    y_val = np.load(DATASET_PATH + "y_val.npy")
    X_test = np.load(DATASET_PATH + "X_test.npy")
    y_test = np.load(DATASET_PATH + "y_test.npy")
    
    train_perm = np.random.permutation(X_train.shape[0])
    X_train, y_train = X_train[train_perm], y_train[train_perm]
    val_perm = np.random.permutation(X_val.shape[0])
    X_val, y_val = X_val[val_perm], y_val[val_perm]
    test_perm = np.random.permutation(X_test.shape[0])
    X_test, y_test = X_test[test_perm], y_test[test_perm]
    
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_val = scaler.fit_transform(X_val)
    X_test = scaler.fit_transform(X_test)
    
    return X_train, y_train, X_val, y_val, X_test, y_test

In [ ]:
def create_model(input_dim, output_dim, params):
    n_layers = params["n_layers"]
    n_neurons = params["n_neurons"]
    dropout = params["dropout"]
    optimizer = params["optimizer"]
    activation = params["activation"]
    last_layer_activation = params["last_layer_activation"]
    loss = params["loss"]
    metrics = params["metrics"]
    
    model = Sequential()
    # Hidden layers
    for n in range(n_layers):
        if n == 0:
            model.add(Dense(n_neurons[n], activation=activation, input_dim=input_dim))
        else:
            model.add(Dense(n_neurons[n], activation=activation))
        if dropout != 0.0:
            model.add(Dropout(dropout))
    # Output layer
    model.add(Dense(output_dim, activation=last_layer_activation))
    
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    
    return model

In [ ]:
def create_emsemble(n_models, instances_ratio, features_ratio, params):
    models = []
    features_idx = []
    for i in range(n_models):
        instances = np.random.choice(range(X_train.shape[0]),
                                     round(X_train.shape[0] * instances_ratio))
        features = np.random.choice(range(X_train.shape[1]),
                                    round(X_train.shape[1] * features_ratio))
        
        xi_train = X_train[instances, features]
        yi_train = y_train[instances]
        xi_val = X_val[:, features]
        
        model = create_model(xi_train.shape[1], yi_train.shape[1], params)
        model.fit(xi_train, yi_train,
                  batch_size=1024,
                  epochs=10000,
                  validation_data=(xi_val, y_val),
                  callbacks=[EarlyStopping(patience=5)],
                  verbose=1)
        model.save_weights(MODELS_PATH + "model-{}.hdf5".format(i), overwrite=True)
        
        models.append(model)
        features_idx.append(features)
        
        # Avoiding memory issues
        K.clear_session()
        
    return models, features_idx

In [ ]:
X_train, y_train, X_val, y_val, X_test, y_test = load_data()

In [ ]:
params = {
    "n_layers": 3,
    "n_neurons": [1024, 1024, 1024],
    "dropout": 0,
    "optimizer": "rmsprop",
    "activation": "sigmoid",
    "last_layer_activation": "sigmoid",
    "loss": "binary_crossentropy",
    "metrics": ["accuracy"],
}